## Homework 02

### Exercise 1
One half percent of the population has a coronavirus and a test is being developed. This test gives a false positive $3\%$ of the time and a false negative $2\%$ of the time. 

1. Find the probability that Luca is positive to the test.
2. Suppose Luca is positive to the test. What is the probability that he has contracted the disease?

#### Proposed solution:
Consider two random variables: $S\in \{0,1\}$, which is 1 if a person has the coronavirus, 0 if he/she doesn't; $T\in \{POS, NEG\}$ which is POS if the test gives positive, NEG if it gives negative.

From the exercise we can say that: $P(S = 1) = 0.005$ (hence $P(S = 0) = 0.995$), moreover we have that $P(T = NEG|S = 1) = 0.02$ (false negative rate) and $P(T = POS|S = 0) = 0.03$ (false positive rate).

1. we have to compute the following marginalization: 

$P(T = POS) = \sum_{S\in \{0,1\}} P(T = POS | S)\cdot P(S) = P(T = POS|S = 0)\cdot P(S = 0) + P(T = POS|S = 1)\cdot P(S = 1)$

Since T is a binary variable, from the law of total probability it must be that $P(T = POS|S = 1) = 1 - P(T = NEG| S = 1) = 1 - 0.02 = 0.98$.

Now we have all the information we need and we plug into the formula:

$P(T = POS) = 0.03\cdot 0.995 + 0.98\cdot 0.005 = 0.03475$

2. we need to compute $P(S = 1|T = POS)$, so we can use the Bayes theorem: 

$P(S = 1|T = POS) = \frac{P(T = POS|S = 1)\cdot P(S = 1)}{P(T = POS)} = \frac{0.98\cdot 0.005}{0.03475} = 0.141$


### Exercise 2

Implement the empirical cumulative distribution function $F_X(x)=$ `cdf(dist, x)` taking as inputs a `pyro.distributions` object `dist`, corresponding to the distribution of $X$, and real value `x`.

Suppose that $X\sim \mathcal{N}(0,1)$ and plot $F_X(x=1)$.

#### Proposed solution:

In [2]:
import torch
import pyro
import matplotlib.pyplot as plt
import numpy as np

loc = 0.
scale = 1. 

dist = pyro.distributions.Normal(loc, scale) #distribution of X ≈ N(0,1)

#trying the pyro tools, to see how they work and to check my function
x1 = torch.linspace(-3.5,1)
f = dist.cdf(x1)

def cdf(dist, x):
    '''
    empirical cumulative distribution function
    '''
    sample = [pyro.sample("normal_sample", dist) for i in range(1000)]
    s = np.sort(sample) 
    sx = [] 
    for i in s:
        if i<x:
            sx.append(i) #sorted since s sorted
    #using arange which returns an evenly spaced array in the given range
    y = np.arange(1, len(sx)+1)/len(s)
    return sx, y

#test and plots
x = 1.
sx, f_x = cdf(dist, x)

plt.plot(x1, f, label = "exact")
plt.plot(sx, f_x, label = "empirical")
plt.xlabel("x")
plt.ylabel("cdf(x)")
plt.title("Empirical CDF of a standard normal distribution")
plt.legend()
plt.show()

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

### Exercise 3

Suppose the heights of male students are normally distributed with mean $180$ and unknown variance $\sigma^2$. Suppose that $\sigma^2$ is in the range $[22,41]$ with approximately $95\%$ probability and assign to $\sigma^2$ an inverse-gamma $IG(38,1110)$ prior distribution .

1. Empirically verify that the parameters of the inverse-gamma distribution lead to a prior probability of approximately $95\%$ that  $\sigma^2\in[22,41]$.
2. Derive the posterior density of $\sigma^2$ corresponding to the following data: $183, 173, 181, 170, 176, 180, 187, 176, 171, 190, 184, 173, 176, 179, 181, 186$.
Then plot it together with the prior density.
3. Compute the posterior density of the standard deviation $\sigma$.

#### Proposed solution: 

In [ ]:
import seaborn as sns
alpha = 38.
beta = 1110.
sigma_prior = pyro.distributions.InverseGamma(alpha, beta)

#1: verify that sigma is in [a,b] = [22,41] with ≈ 95% probability

a = 22.
b = 41.

sample_prior = [pyro.sample("normal_sample", sigma_prior) for i in range(1000)]
s = []
for i in sample_prior:
    if i>=a and i<=b:
        s.append(i)
prop = (len(s)*100)/len(sample_prior)
print("sigma in [22,41] with estimated probability (using sample): ", prop)

#we can show the same using the function defined in the previous exercise
sa, sigma_cdf_a = cdf(sigma_prior, a)
sb, sigma_cdf_b = cdf(sigma_prior, b)

prop_cdf = sigma_cdf_b[-1].item() - sigma_cdf_a[-1].item()
print("sigma in [22,41] with estimated probability (using cdf): ", prop_cdf)

#2: derived posterior density of sigma^2 using some observation
loc = 180.
#these data come from a normal distribution N(180, sigma^2) (likelihood)
data = [183,173,181,170,176,180,187,176,171,190,184,173,176,179,181,186]

#from data we can estimate the sample variance and the sample mean
def sample_mean(data):
    s_mean = 0
    for d in data:
        s_mean+=d
    s_mean /= len(data)
    return s_mean

def sample_variance(data):
    s_mean = sample_mean(data)
    s_var = 0
    for d in data:
        s_var+=((d - s_mean)**2)
    s_var /= (len(data)-1)
    return s_var

scale = sample_variance(data)
likelihood = pyro.distributions.Normal(loc, scale)

sample_likelihood = [pyro.sample("normal_sample", likelihood) for i in range(1000)]


estimated_posterior_unscaled = []
for i in range(0,len(sample_prior)): #Bayes theorem
    estimated_posterior_unscaled.append(sample_likelihood[i]*sample_prior[i])
    
#now we need to scale the estimated_posterior --> compute the normalization factor
#we can use the function (slightly adapted) defined in notebook 2
def scale(guess):
    height = pyro.sample("weight", pyro.distributions.Normal(guess, 1.0))
    measurement = [pyro.sample("measurement", pyro.distributions.Normal(height, 0.75)) for i in range(1000)]
    return measurement

norm_factor = scale(sample_mean(data))
estimated_posterior = []
for i in range(len(estimated_posterior_unscaled)):
    estimated_posterior.append(estimated_posterior_unscaled[i]/norm_factor[i])
sns.distplot(estimated_posterior,  kde_kws={"label": "posterior"})
sns.distplot(sample_prior, kde_kws={"label": "prior"})


3. posterior density of $\sigma$

We have that the likelihood is: $p(x|\mu, \sigma^2) = \frac{1}{\sqrt{2\pi \sigma^2}}\cdot e^{-\frac{(x-\mu)^2}{2 \sigma^2}} $, namely $\mathcal{N}(\mu, \sigma^2)$, with known mean $\mu$.

While the prior is: $p(\sigma^2) = \frac{\beta^{\alpha}}{\Gamma(\alpha)}\cdot (\sigma^2)^{-(\alpha + 1)}\cdot e^{-\frac{\beta}{\sigma^2}}$, namely a $IG(\alpha, \beta)$, with both parameter known

The posterior $p(\sigma^2|x, \mu)$ is such that $p(\sigma^2|x, \mu)\propto p(x|\mu, \sigma^2)\cdot p(\sigma^2)$.
Compute the right hand side of the previous expression, ignoring - for the moment - constants wrt $\sigma^2$.

$(\sigma^2)^{-\frac{1}{2}}\cdot e^{-\frac{1}{\sigma^2}\cdot \frac{(x-\mu)^2}{2}}\cdot (\sigma^2)^{-(\alpha+1)}\cdot e^{-\frac{\beta}{\sigma^2}} = (\sigma^2)^{-(\alpha+\frac{1}{2}+1)}\cdot e^{-\frac{1}{\sigma^2}\cdot (\beta + \frac{(x-\mu)^2}{2})}$
 
We obtained an $IG(\alpha+\frac{1}{2}, \beta+\frac{(x-\mu)^2}{2})$. 

We plug this new parameters into the previously ignored constants, and putting all together we get the posterior:

$p(\sigma^2|x,\mu) =\frac{((\beta + \frac{(x-\mu)^2}{2}))^{\alpha + \frac{1}{2}}}{\Gamma(\alpha+\frac{1}{2})} (\sigma^2)^{-(\alpha+\frac{1}{2}+1)}\cdot e^{-\frac{1}{\sigma^2}\cdot (\beta + \frac{(x-\mu)^2}{2})}$

Since we also know that $\alpha = 38, \ \beta = 1110, \ \mu = 180$, we can further substitute, getting the posterior density of $\sigma$:

$p(\sigma|x) = \frac{((1110 + \frac{(x-180)^2}{2}))^{38.5}}{\Gamma(38.5)} (\sigma^2)^{-39.5}\cdot e^{-\frac{1}{\sigma^2}\cdot (1110 + \frac{(x-180)^2}{2})}$

### Exercise 4

Prove that the Gamma distribution is the conjugate prior distribution for the Exponential likelihood.

#### Proposed solution:

Consider an exponential likelihood: $p(x|\lambda) = \lambda\cdot e^{-\lambda\cdot x}$, and a Gamma prior $p(\lambda) = \frac{\beta^{\alpha}}{\Gamma(\alpha)}\cdot \lambda^{\alpha-1}\cdot e^{-\beta\cdot \lambda}$.

From Bayes theorem we have that the posterior $p(\lambda|x)\propto p(x|\lambda)\cdot p(\lambda)$.

We compute the right hand side of the previous expression ignoring the constants wrt $\lambda$:

$\lambda\cdot e^{-\lambda\cdot x}\cdot \lambda^{\alpha-1}\cdot e^{-\beta\cdot \lambda} = \lambda^{\alpha}\cdot e^{-\lambda\cdot (\beta + x)}$ which is a $Gamma(\alpha+1, \beta+x)$.

Hence the Gamma distribution is the conjugate prior distribution for the Exponential likelihood.